In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pydub

In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
import os
from pydub import AudioSegment
from datasets import Dataset, DatasetDict, load_dataset, Audio
import pandas as pd

In [ ]:
def convert_mp3_to_wav(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith(".mp3"):
            mp3_path = os.path.join(input_dir, filename)
            wav_path = os.path.join(output_dir, filename.replace(".mp3", ".wav"))

            # Convert MP3 to WAV
            audio = AudioSegment.from_mp3(mp3_path)
            audio = audio.set_frame_rate(16000)  # Set sample rate to 16 kHz
            audio.export(wav_path, format="wav")
            print(f"Converted {filename} to {wav_path}")

# Convert MP3 to WAV
convert_mp3_to_wav("/content/drive/MyDrive/Audio_Processed", "/content/drive/MyDrive/ConvertedAP")

Converted Audio_Processedprocessed_SandalWoodNewsStories_144.mp3 to /content/drive/MyDrive/ConvertedAP/Audio_Processedprocessed_SandalWoodNewsStories_144.wav
Converted Audio_Processedprocessed_SandalWoodNewsStories_63.mp3 to /content/drive/MyDrive/ConvertedAP/Audio_Processedprocessed_SandalWoodNewsStories_63.wav
Converted Audio_Processedprocessed_SandalWoodNewsStories_1.mp3 to /content/drive/MyDrive/ConvertedAP/Audio_Processedprocessed_SandalWoodNewsStories_1.wav
Converted Audio_Processedprocessed_SandalWoodNewsStories_257.mp3 to /content/drive/MyDrive/ConvertedAP/Audio_Processedprocessed_SandalWoodNewsStories_257.wav
Converted Audio_Processedprocessed_SandalWoodNewsStories_291.mp3 to /content/drive/MyDrive/ConvertedAP/Audio_Processedprocessed_SandalWoodNewsStories_291.wav
Converted Audio_Processedprocessed_SandalWoodNewsStories_181.mp3 to /content/drive/MyDrive/ConvertedAP/Audio_Processedprocessed_SandalWoodNewsStories_181.wav
Converted Audio_Processedprocessed_SandalWoodNewsStories_1

In [9]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-vlqkq9_q
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-vlqkq9_q
  Resolved https://github.com/openai/whisper.git to commit 5979f03701209bb035a0a466f14131aeb1116cbb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803560 sha256=094203542c5f03fd5b737d9a81e4e053334b81093fb824457677321a20070ad3
  Stored in directory: /tmp/pip-ephem-wheel-cache-0368754c/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [5]:
!pip install speechrecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 49.1 MB/s eta 0:00:00


In [10]:
import speech_recognition as sr

# Define paths
wav_dir = "/content/drive/MyDrive/ConvertedAP"
output_csv = "/content/drive/MyDrive/transcriptions.csv"

# Initialize recognizer
recognizer = sr.Recognizer()

# Load existing CSV or create a new DataFrame if it doesn't exist
if os.path.exists(output_csv):
    transcriptions_df = pd.read_csv(output_csv)
    processed_files = set(transcriptions_df['file'].tolist())  # Set of already processed files
else:
    transcriptions_df = pd.DataFrame(columns=["file", "transcription"])
    processed_files = set()

# Process each audio file in the directory
for filename in os.listdir(wav_dir):
    if filename.endswith(".wav") and filename not in processed_files:
        file_path = os.path.join(wav_dir, filename)

        # Load the audio file
        with sr.AudioFile(file_path) as source:
            audio = recognizer.record(source)  # Read the entire audio file

        try:
            # Perform speech-to-text using Google Speech Recognition
            transcription = recognizer.recognize_google(audio, language="kn-IN")  # Use "kn-IN" for Kannada

            # Append the filename and transcription to the DataFrame using pd.concat
            new_row = pd.DataFrame({"file": [filename], "transcription": [transcription]})
            transcriptions_df = pd.concat([transcriptions_df, new_row], ignore_index=True)
            print(f"Transcribed {filename}: {transcription}")

        except sr.UnknownValueError:
            print(f"Google Speech Recognition could not understand audio in {filename}")
            new_row = pd.DataFrame({"file": [filename], "transcription": ["Unintelligible"]})
            transcriptions_df = pd.concat([transcriptions_df, new_row], ignore_index=True)

        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service for {filename}; {e}")
            new_row = pd.DataFrame({"file": [filename], "transcription": ["Error"]})
            transcriptions_df = pd.concat([transcriptions_df, new_row], ignore_index=True)

        # Save progress after each file
        transcriptions_df.to_csv(output_csv, index=False)

print("Batch processing completed. Transcriptions saved to CSV.")

Google Speech Recognition could not understand audio in Audio_Processedprocessed_SandalWoodNewsStories_144.wav
Google Speech Recognition could not understand audio in Audio_Processedprocessed_SandalWoodNewsStories_63.wav
Could not request results from Google Speech Recognition service for Audio_Processedprocessed_SandalWoodNewsStories_1.wav; recognition connection failed: [Errno 32] Broken pipe
Google Speech Recognition could not understand audio in Audio_Processedprocessed_SandalWoodNewsStories_257.wav
Transcribed Audio_Processedprocessed_SandalWoodNewsStories_291.wav: ಬೆಂಗಳೂರು ನಗರ ಪೊಲೀಸ್ ಕಮಿಷನರ್ ಕಚೇರಿಯ ಕೂಗಳತೆ ದೂರದಲ್ಲಿ ಕಳ್ಳತನ ನಡೆದಿದೆ ಮಚ್ಚು ದೊಣ್ಣೆಗಳಿಂದ ಭದ್ರತಾ ಸಿಬ್ಬಂದಿಯನ್ನು ಬೆದರಿಸಿ ಎಂಎಸ್ಸಿ ಅಪಾರ್ಟ್ಮೆಂಟ್ನಲ್ಲಿ ಗಂಧದ ಮರವನ್ನು ಕಳ್ಳರು ಕಥೆಗಳು 21ನೇ ತಾರೀಕು ಘಟನೆ ನಡೆದಿದ್ದು ತಡವಾಗಿ ಬೆಳಕಿಗೆ ಬಂದಿದೆ ವಿಧಾನಸೌಧ ಠಾಣೆಯಲ್ಲಿ ಪ್ರಕರಣ ದಾಖಲಾಗಿದ್ದು ಪೊಲೀಸರು ತನಿಖೆ ನಡೆಸುತ್ತಿದ್ದಾರೆ
Transcribed Audio_Processedprocessed_SandalWoodNewsStories_181.wav: ಫ್ರಾನ್ಸಿಸ್ ಫಿಫ್ಟಿ ಪರ್ಸೆಂಟ್ ಕಟ್ ಮಾಡ್ತಾ ಇರೋದು ಅಂತ ಹೇಳಿ ಪೂರ್ತಿ ಏನ್ ಸಮಸ್ಯೆ ಇ